<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/LearningWithoutForgetting2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# upload work files from your git hub repository
import sys
!rm -r IncrementalLearning
!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 274, done.
remote: Total 274 (delta 0), reused 0 (delta 0), pack-reused 274
Receiving objects: 100% (274/274), 144.75 KiB | 6.89 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [4]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb
import copy
import pickle
import pandas as pd

# project classes --------------------------------------------------------------
from IncrementalLearning.cifar100 import ilCIFAR100
from resnet_cifar import resnet32

In [11]:
class lwf():
  def __init__(self, random_seed, batch_size):
    
    # hyper parameters related attributes
    self.batch_size = batch_size
    self.classes_per_task = 10
    self.num_tasks = 10
    self.LR = 2
    self.MOMENTUM = 0.9
    self.WEIGHT_DECAY = 1e-5
    self.MILESTONES = [49,63]
    self.GAMMA = 0.2
    self.numepochs = 70
    
    # dataset related attributes
    self.random_seed = random_seed
    self.original_training_set = ilCIFAR100(self.classes_per_task, random_seed)
    self.original_test_set = ilCIFAR100(self.classes_per_task, random_seed, train=False)
    
    # models related attributes
    self.model = resnet32(num_classes=100).to('cuda')
    self.criterion = nn.BCEWithLogitsLoss()
    self.mapper = self.original_training_set.get_dict()
    self.current_classes = 0

  def update_parameters(self, train_dataloader):
    
    # update current_classes
    self.current_classes += self.classes_per_task

    # initialize optimizer and scheduler
    optimizer = optim.SGD(self.model.parameters(), lr=self.LR, momentum=self.MOMENTUM, weight_decay=self.WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.MILESTONES, gamma=self.GAMMA)

    # create old _model and set in evaluation mode
    old_model = copy.deepcopy(self.model)
    old_model = old_model.to('cuda')
    old_model.eval()

    # set current model in training mode
    self.model.train()
  
    for epoch in range(self.numepochs):
      for inputs, labels in train_dataloader:
        
        # read data batch inputs and map original labels to new labels
        labels = torch.tensor([torch.tensor(self.mapper[c.item()]) for c in labels])
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        # zeroing gradients
        optimizer.zero_grad()
        
        # compute net outputs 
        outputs = self.model(inputs)
        
        # set up output targets
        target = torch.eye(100)[labels] 
        target = target.to('cuda')

        # update target tensor if you have more than ten classes activated (since second task)
        if self.current_classes != self.classes_per_task:
          known_classes = self.current_classes - self.classes_per_task
          old_target = old_model(inputs).to('cuda')
          old_target = torch.sigmoid(old_target).to('cuda')
          target = torch.cat((old_target[:,:known_classes], target[:,known_classes:]), dim=1)
        
        # compute loss
        loss=self.criterion(outputs, target)
        
        # update gradients
        loss.backward()
        optimizer.step()

      # update scheduler
      scheduler.step()

  
  def compute_accuracy(self, dataloader):
    total = 0.0
    running_corrects = 0.0
    for images, labels in dataloader:
      total += len(labels)
      labels = torch.tensor([torch.tensor(self.mapper[c.item()]) for c in labels])
      images = images.to('cuda')
      labels = labels.to('cuda')
      outputs = self.model(images)
      _, preds = torch.max(outputs.data, 1)
      running_corrects += torch.sum(preds == labels.data).data.item()
    accuracy = running_corrects / total
    return accuracy

  def training(self):
    
    # get train and test indixes
    train_indexes= self.original_training_set.get_batch_indexes()
    test_indexes = self.original_test_set.get_batch_indexes()
    current_test_indexes=[]

    # initialize accuracy related variables
    accuracy = 0 
    tasks_test_acc = []

    for i in range(self.num_tasks):
      
      # make current train and test datasets 
      train_dataset = Subset(self.original_training_set, train_indexes[i])
      current_test_indexes += test_indexes[i].tolist()
      test_dataset = Subset(self.original_test_set, current_test_indexes)
      
      # make current train and test dataloaders 
      train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=False)
      test_dataloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)        
      
      # train model for current task
      self.update_parameters(train_dataloader)
      
      # set model in evaluation mode
      self.model.eval()

      # LAST EPOCH MODEL TRAIN ACCURACY
      train_acc = self.compute_accuracy(train_dataloader)
      
      # TEST ACCURACY
      test_acc = self.compute_accuracy(test_dataloader)
      print('TASK %d    TRAIN_ACC: %.2f    TEST_ACC: %.2f' % (i, 100*train_acc, 100*test_acc))
      
      # save current test accuracy
      tasks_test_acc.append(accuracy)

    with open("Lwf_"+str(self.random_seed), "wb") as file:
      pickle.dump(tasks_test_acc, file, pickle.HIGHEST_PROTOCOL)
    

In [12]:
seed = 981
print("NEW RUN, seed %d"%(seed))
lwf = lwf(batch_size=128, random_seed=seed)
lwf.training()

"""SEED = [981, 404, 182]
for i in range(3):
  print("NEW RUN, seed %d"%(SEED[i]))
  model = lwf(batch_size = 128, random_seed = SEED[i])
  model.train_model()"""

NEW RUN, seed 981
Files already downloaded and verified
Files already downloaded and verified
TASK 0    TRAIN_ACC: 99.12    TEST_ACC: 84.80
TASK 1    TRAIN_ACC: 94.06    TEST_ACC: 67.90
TASK 2    TRAIN_ACC: 93.98    TEST_ACC: 56.47
TASK 3    TRAIN_ACC: 91.14    TEST_ACC: 47.90
TASK 4    TRAIN_ACC: 90.60    TEST_ACC: 42.14
TASK 5    TRAIN_ACC: 87.56    TEST_ACC: 38.17
TASK 6    TRAIN_ACC: 87.10    TEST_ACC: 32.73
TASK 7    TRAIN_ACC: 88.32    TEST_ACC: 29.40
TASK 8    TRAIN_ACC: 85.66    TEST_ACC: 26.86
TASK 9    TRAIN_ACC: 84.06    TEST_ACC: 22.74


'SEED = [981, 404, 182]\nfor i in range(3):\n  print("NEW RUN, seed %d"%(SEED[i]))\n  model = lwf(batch_size = 128, random_seed = SEED[i])\n  model.train_model()'

In [ ]:
"""import seaborn as sns
import matplotlib.pyplot as plt

test_acc = []
for seed in SEED:
  with open('Lwf_'+str(seed), 'rb') as f:
    current = pickle.load(f)
    test_acc += current

acc_dict = {'model': ['lwf' for i in range(30)], 
            'classes': [i for i in range(10,110,10)]*3,
            'test_acc': test_acc}
            
acc_df = pd.DataFrame(data=acc_dict)

sns.set_theme(style="darkgrid")
ax = sns.pointplot(x="classes", y="test_acc", hue="model", data=acc_df)"""